In [229]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [266]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
Postcode = []
Borough = []
Neighbourhood = []
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))       
data =  pd.DataFrame({'Postalcode':Postcode,'Borough':Borough,'Neighbourhood':Neighbourhood})

In [268]:
data.head(10)
data["Neighbourhood"].replace("Not assigned\n","Not assigned",inplace = True)

In [277]:
#Datafrme
#Ignore cells with a borough that is Not assigned....
df = data.drop(data.index[data["Borough"]=="Not assigned"])

In [278]:
#More than one neighborhood can exist in one postal code area...
duplicated_Postalcode = df[df.duplicated(['Postalcode'])]["Postalcode"]
for i in range(len(duplicated_Postalcode)):
    temp_values = df["Neighbourhood"][df['Postalcode']==duplicated_Postalcode.iloc[i]].values.tolist()
    df["Neighbourhood"][df['Postalcode']==duplicated_Postalcode.iloc[i]] = ",".join(temp_values)
df = df.drop_duplicates()

In [282]:
df["Neighbourhood"][df["Neighbourhood"]=="Not assigned"]=df["Borough"][df["Neighbourhood"]=="Not assigned"]
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson\n,Garden District\n"


In [283]:
print(np.shape(df))

(103, 3)
